In [58]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

In [59]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20201005


In [60]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

ディレクトリ

In [61]:
data_dir = Path(r"C:\Users\crie072\Documents\21_Online")

Inputファイル

In [62]:
#セキュポスデータ from CSC 期中と期末２ファイルやる
input_file1 = "N029（2020年9月1日～23日入会）.xlsx"

#デジガレ確認用リスト
input_file2 = "20200925〆【JFRカード御中】成果データ&問い合わせ.xlsx"

Outputファイル

In [63]:
output_file1 = f"オンライン獲得レポート集計用データ（通常）_{20200928}"
print(output_file1)

output_file2 = f"オンライン獲得レポート集計用データ（利用条件付き）_{20200928}"
print(output_file2)

オンライン獲得レポート集計用データ（通常）_20200928
オンライン獲得レポート集計用データ（利用条件付き）_20200928


セキュポスデータ取込

In [64]:
df = pd.read_excel(data_dir /r"input"/f"{input_file1}", dtype={'エンボス内容': str})
df['作業日'] = today_yyyymmdd
df.head()

,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD,作業日
0,20200901,1,0,D00,8,31,1,0,8,107,2,0,NaN,NaN,NaN,NaN,155203017878797,55,12.0,7000,20.0,07000401365,2,0,20201005
1,20200901,1,1,D02,1,2,1,0,3,103,2,0,NaN,NaN,NaN,NaN,155603602494324,53,12.0,3000,NaN,03000010216,2,0,20201005
2,20200901,1,0,D00,1,31,1,0,8,102,2,0,NaN,NaN,NaN,NaN,155203017879712,48,12.0,2000,20.0,02000410019,2,0,20201005
3,20200901,1,0,D00,1,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921991085,38,52.0,10000,40.0,10000,2,1,20201005
4,20200901,1,0,D00,4,31,2,0,3,299,1,0,NaN,NaN,NaN,NaN,155051922005075,51,12.0,11000,40.0,11000,2,0,20201005


In [65]:
#オンライン入会に限定する
df = df[(df["会員カード発行区分"] == "1") 
               & (df["続柄"] == 0) 
               & ~(df["カード提携会社2CD"] == 95) 
               & (df["切替CD"] == 0) 
               & (df["受付部支店"] == 299)]
df.shape[0]

577

In [66]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(577, 25)
Int64Index([   3,    4,    7,   15,   18,   28,   32,   34,   36,   38,
            ...
            3944, 3945, 3946, 3947, 3948, 3949, 3951, 3960, 4002, 4057],
           dtype='int64', length=577)
Index(['カード加入日', '会員カード発行区分', '続柄', 'カード提携会社1CD', 'カード提携会社2CD', 'カード種類CD',
       'カードブランドCD', '切替CD', '勧誘CD', '受付部支店', '性別CD', '再発行CD', '旧カード提携会社1CD',
       '旧カード提携会社2CD', '旧カード種類CD', '旧カードブランドCD', '入会申込番号', '満年齢', 'リボ支払方法CD',
       '勧誘団体CD', '提携カード特殊エリア', 'エンボス内容', 'カード年会費CD', 'リボ利用対象CD', '作業日'],
      dtype='object')
カード加入日           int64
会員カード発行区分       object
続柄               int64
カード提携会社1CD      object
カード提携会社2CD       int64
カード種類CD          int64
カードブランドCD        int64
切替CD             int64
勧誘CD             int64
受付部支店            int64
性別CD             int64
再発行CD            int64
旧カード提携会社1CD     object
旧カード提携会社2CD    float64
旧カード種類CD       float64
旧カードブランドCD     float64
入会申込番号           int64
満年齢              int64
リボ支払方法CD       float64
勧誘団体CD           int64
提携カード特殊

In [67]:
df['カード加入日'] = pd.to_datetime(df['カード加入日'], format = '%Y%m%d')
df['カ－ド加入年'] = df['カード加入日'].dt.year
df['カ－ド加入月'] = df['カード加入日'].dt.month

#"加入年月"のカラム生成
df['カ－ド加入年月'] = df[['カ－ド加入年', 'カ－ド加入月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df.head()

,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD,作業日,カ－ド加入年,カ－ド加入月,カ－ド加入年月
3,2020-09-01,1,0,D00,1,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921991085,38,52.0,10000,40.0,10000,2,1,20201005,2020,9,2020年9月
4,2020-09-01,1,0,D00,4,31,2,0,3,299,1,0,NaN,NaN,NaN,NaN,155051922005075,51,12.0,11000,40.0,11000,2,0,20201005,2020,9,2020年9月
7,2020-09-01,1,0,D00,1,31,1,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921984189,46,12.0,6000,40.0,06000,2,0,20201005,2020,9,2020年9月
15,2020-09-01,1,0,D00,1,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921991556,56,12.0,1000,41.0,01000,2,0,20201005,2020,9,2020年9月
18,2020-09-01,1,0,D00,4,31,2,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921990830,72,12.0,2000,40.0,02000,2,0,20201005,2020,9,2020年9月


In [68]:
#入会申込番号の重複がないかチェック
print(len(df) == len(df['入会申込番号'].unique()))

True


In [69]:
#重複削除
df = df.drop_duplicates(subset=['入会申込番号'])
df.shape[0]

577

In [70]:
df=df[['入会申込番号','カード加入日','カ－ド加入年月','作業日']]
df.head()

,入会申込番号,カード加入日,カ－ド加入年月,作業日
3,155051921991085,2020-09-01,2020年9月,20201005
4,155051922005075,2020-09-01,2020年9月,20201005
7,155051921984189,2020-09-01,2020年9月,20201005
15,155051921991556,2020-09-01,2020年9月,20201005
18,155051921990830,2020-09-01,2020年9月,20201005


In [71]:
#欠損データがないかチェック
df.isnull().sum()

入会申込番号     0
カード加入日     0
カ－ド加入年月    0
作業日        0
dtype: int64

In [17]:
#df.to_csv(data_dir /r"output"/f"{output_file1}.csv", index=False,header=True,encoding='utf-8-sig')

デジガレのリスト読込

In [72]:
print(input_file2)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file2}", sheet_name = 0)
df_dg = df_dg.rename(columns = {'参照用': '入会申込番号'})

#シート名取得
bk = pd.ExcelFile(data_dir/r"input"/f"{input_file2}")
print(bk.sheet_names)

df_dg.head()

20200925〆【JFRカード御中】成果データ&問い合わせ.xlsx
['成果データ', '識別子不明', '成果データ（5月～利用条件追加媒体）', '成果データ（5月～利用条件追加媒体）_承認返却済み']


,ASID,メディア名,媒体チャネル,クリック,申込,申込月,入会申込番号,ASP名,単価,成果,カード発行日
0,49.0,ハピタス（連携用）,ポイント,2020-05-28 12:54:13,2020-05-28 13:11:05,43952,155051921900045,BGT,7000,保留,保留
1,49.0,ハピタス（連携用）,ポイント,2020-05-28 13:52:57,2020-05-28 14:02:05,43952,155051921900110,BGT,7000,保留,保留
2,619.0,モッピー（連携用）,ポイント,2020-05-28 14:24:02,2020-05-28 14:38:04,43952,155051921900169,BGT,7000,保留,保留
3,49.0,ハピタス（連携用）,ポイント,2020-05-28 14:36:08,2020-05-28 14:45:04,43952,155051921900185,BGT,7000,保留,保留
4,619.0,モッピー（連携用）,ポイント,2020-05-28 14:41:20,2020-05-28 14:51:06,43952,155051921900201,BGT,7000,保留,保留


In [73]:
print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(1366, 11)
RangeIndex(start=0, stop=1366, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '申込月', '入会申込番号', 'ASP名', '単価',
       '成果', 'カード発行日'],
      dtype='object')
ASID             float64
メディア名             object
媒体チャネル            object
クリック              object
申込        datetime64[ns]
申込月                int64
入会申込番号             int64
ASP名              object
単価                 int64
成果                object
カード発行日            object
dtype: object


In [74]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,入会申込番号,ASP名,単価,成果,カード発行日,申込年月
0,49.0,ハピタス（連携用）,ポイント,2020-05-28 12:54:13,2020-05-28 13:11:05,155051921900045,BGT,7000,保留,保留,2020年5月
1,49.0,ハピタス（連携用）,ポイント,2020-05-28 13:52:57,2020-05-28 14:02:05,155051921900110,BGT,7000,保留,保留,2020年5月
2,619.0,モッピー（連携用）,ポイント,2020-05-28 14:24:02,2020-05-28 14:38:04,155051921900169,BGT,7000,保留,保留,2020年5月
3,49.0,ハピタス（連携用）,ポイント,2020-05-28 14:36:08,2020-05-28 14:45:04,155051921900185,BGT,7000,保留,保留,2020年5月
4,619.0,モッピー（連携用）,ポイント,2020-05-28 14:41:20,2020-05-28 14:51:06,155051921900201,BGT,7000,保留,保留,2020年5月


In [75]:
df_dg.shape[0]

1366

In [76]:
pd.crosstab(df_dg['申込年月'],df_dg['媒体チャネル'])

媒体チャネル,ポイント,比較
申込年月,,
2020年1月,3,0
2020年2月,2,0
2020年5月,34,0
2020年6月,120,3
2020年7月,128,6
2020年8月,127,6
2020年9月,931,6


In [77]:
df_dg = df_dg[['入会申込番号','媒体チャネル','単価','申込年月']]

In [78]:
#2つのテーブルをマージ
df_dg2 = pd.merge(df,df_dg, on =['入会申込番号'], how = 'left')
df_dg2.shape[0]

577

In [79]:
df_dg2.head()

,入会申込番号,カード加入日,カ－ド加入年月,作業日,媒体チャネル,単価,申込年月
0,155051921991085,2020-09-01,2020年9月,20201005,NaN,NaN,NaN
1,155051922005075,2020-09-01,2020年9月,20201005,ポイント,9000.0,2020年9月
2,155051921984189,2020-09-01,2020年9月,20201005,NaN,NaN,NaN
3,155051921991556,2020-09-01,2020年9月,20201005,NaN,NaN,NaN
4,155051921990830,2020-09-01,2020年9月,20201005,NaN,NaN,NaN


集計

In [80]:
#客数
df_count = df_dg2.groupby(['作業日','カ－ド加入年月','媒体チャネル','申込年月','単価'])['入会申込番号'].count()
df_count.to_csv(data_dir /r"output"/f"{output_file1}.csv", sep=',',header=True,encoding='utf-8-sig')

df_count.head()

作業日       カ－ド加入年月  媒体チャネル  申込年月     単価    
20201005  2020年9月  ポイント    2020年7月  7000.0     11
                           2020年8月  7000.0     32
                           2020年9月  7000.0    174
                                    9000.0    140
Name: 入会申込番号, dtype: int64

利用条件

In [27]:
print(input_file2)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file2}", sheet_name = 2)
df_dg = df_dg.rename(columns = {'参照用': '入会申込番号'})

#シート名取得
bk = pd.ExcelFile(data_dir/r"input"/f"{input_file2}")
print(bk.sheet_names)

df_dg.head()

20200925〆【JFRカード御中】成果データ&問い合わせ.xlsx
['成果データ', '識別子不明', '成果データ（5月～利用条件追加媒体）', '成果データ（5月～利用条件追加媒体）_承認返却済み']


,ASID,メディア名,媒体チャネル,クリック,申込,入会申込番号,ASP名,単価,成果,カード発行日,カード売上額合計
0,21.0,ポイントインカム,ポイント,2020-06-01 01:25:55,2020-06-01 01:34:29,155051921904849,BGT,9000,保留,保留,1.550519e+14
1,128.0,Warau.jp,ポイント,2020-06-01 09:42:13,2020-06-01 09:50:12,155051921904930,BGT,9000,保留,保留,1.550519e+14
2,5954.0,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-06-01 12:35:04,2020-06-01 12:58:12,155051921905143,BGT,9000,保留,保留,1.550519e+14
3,57.0,ライフメディア,ポイント,2020-06-01 16:52:21,2020-06-01 17:03:12,155051921905432,BGT,9000,保留,20200624,1.550519e+14
4,21.0,ポイントインカム,ポイント,2020-06-01 22:16:51,2020-06-01 22:26:12,155051921905846,BGT,9000,保留,保留,1.550519e+14


In [28]:
print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(918, 11)
RangeIndex(start=0, stop=918, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '入会申込番号', 'ASP名', '単価', '成果',
       'カード発行日', 'カード売上額合計'],
      dtype='object')
ASID               float64
メディア名               object
媒体チャネル              object
クリック        datetime64[ns]
申込          datetime64[ns]
入会申込番号               int64
ASP名                object
単価                   int64
成果                  object
カード発行日              object
カード売上額合計           float64
dtype: object


In [29]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,入会申込番号,ASP名,単価,成果,カード発行日,カード売上額合計,申込年月
0,21.0,ポイントインカム,ポイント,2020-06-01 01:25:55,2020-06-01 01:34:29,155051921904849,BGT,9000,保留,保留,1.550519e+14,2020年6月
1,128.0,Warau.jp,ポイント,2020-06-01 09:42:13,2020-06-01 09:50:12,155051921904930,BGT,9000,保留,保留,1.550519e+14,2020年6月
2,5954.0,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-06-01 12:35:04,2020-06-01 12:58:12,155051921905143,BGT,9000,保留,保留,1.550519e+14,2020年6月
3,57.0,ライフメディア,ポイント,2020-06-01 16:52:21,2020-06-01 17:03:12,155051921905432,BGT,9000,保留,20200624,1.550519e+14,2020年6月
4,21.0,ポイントインカム,ポイント,2020-06-01 22:16:51,2020-06-01 22:26:12,155051921905846,BGT,9000,保留,保留,1.550519e+14,2020年6月


In [30]:
df_dg.shape[0]

918

In [31]:
df_dg = df_dg[['入会申込番号','媒体チャネル','単価','申込年月']]

In [32]:
#2つのテーブルをマージ
df_dg2 = pd.merge(df,df_dg, on =['入会申込番号'], how = 'left')
df_dg2.shape[0]

793

In [33]:
df_dg2.head()

,入会申込番号,カード加入日,カ－ド加入年月,作業日,媒体チャネル,単価,申込年月
0,155051922001363,2020-09-01,2020年9月,20201005,NaN,NaN,NaN
1,155051922005174,2020-09-01,2020年9月,20201005,NaN,NaN,NaN
2,155051922005075,2020-09-01,2020年9月,20201005,NaN,NaN,NaN
3,155051921988313,2020-09-01,2020年9月,20201005,ポイント,9000.0,2020年8月
4,155051921993446,2020-09-01,2020年9月,20201005,NaN,NaN,NaN


In [34]:
#客数
df_count = df_dg2.groupby(['作業日','カ－ド加入年月','媒体チャネル','申込年月','単価'])['入会申込番号'].count()
df_count.to_csv(data_dir /r"output"/f"{output_file2}.csv", header=True,encoding='utf-8-sig')

df_count.head()

作業日       カ－ド加入年月  媒体チャネル  申込年月     単価    
20201005  2020年9月  ポイント    2020年6月  9000.0     1
                           2020年7月  9000.0    11
                           2020年8月  9000.0    44
Name: 入会申込番号, dtype: int64